### Execute `!nvidia-smi` to check GPU availability.

In [ ]:
!nvidia-smi

### Import necessary libraries

In [ ]:
import os
import nibabel as nib
import numpy as np 
from collections import OrderedDict
import json
from pathlib import Path

from utils.helper import convert_nrrd_to_nifti, create_folder, plot_all_slices, plot_histogram, generate_binary_image, adjust_affine_for_spacing_and_origin, save_binary_image_with_adjusted_origin, make_if_dont_exist
from utils.metrics import dice_score_per_class, hausdorff_distance_per_class, ravd_per_class

### Define dataset paths

In [ ]:
# define dataset path
BASE_PATH = Path('./').resolve()
DATA_PATH = BASE_PATH / 'dataset'

project_name = 'HPC' #change here for different task name
task_name = 'Dataset003_' + project_name 

TRAINING_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'imagesTr'
GT_TRAINING_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'labelsTr'
TEST_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'imagesTs'
GT_TEST_DATASET_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name / 'labelsTs'
PREDICTION_RESULTS_PATH  = BASE_PATH / 'dataset/nnUNet_Prediction_Results' / task_name
TASK_PATH = BASE_PATH / 'dataset/nnUNet_raw_data' / task_name 

# setup environment variables
nnUNet_raw = BASE_PATH / 'dataset/nnUNet_raw_data'
nnUNet_preprocessed = BASE_PATH / 'dataset/nnUNet_preprocessed'
nnUNet_results = BASE_PATH / 'dataset/nnUNet_results'

In [ ]:
train_files = os.listdir(TRAINING_DATASET_PATH)
label_files = os.listdir(GT_TRAINING_DATASET_PATH)
print("train image files:",len(train_files))
print("train label files:",len(label_files))
print("Matches:",len(set(train_files).intersection(set(label_files))))


### nnU-Net v2 installtion 

In [ ]:
pip install nnunetv2

### Clone nnU-Net from git repo

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git

### Install hiddenlayer

pip install --upgrade git+https://github.com/FabianIsensee/hiddenlayer.git

### Install packages

In [ ]:
os.chdir(BASE_PATH)
os.getcwd()

os.chdir('nnUNet')
os.getcwd()

pip install -e .

os.chdir(BASE_PATH)
os.getcwd()

## Set up environment variables

Setup check ``{echo $nnUNet_raw, echo $nnUNet_preprocessed, echo $nnUNet_results}`` 

In [ ]:
%env nnUNet_raw=$nnUNet_raw
%env nnUNet_preprocessed=$nnUNet_preprocessed
%env nnUNet_results=$nnUNet_results

## Plan  and preprocess

In [ ]:
!nnUNetv2_plan_and_preprocess -d 3 --verify_dataset_integrity

### Train 2d 

In [ ]:
!nnUNetv2_train 3 2d 0 --npz 

In [ ]:
!nnUNetv2_train 3 2d 1 --npz 

In [ ]:
!nnUNetv2_train 3 2d 2 --npz 

In [ ]:
!nnUNetv2_train 3 2d 3 --npz 

In [ ]:
!nnUNetv2_train 3 2d 4 --npz 

### Train 3d lower resolution

In [ ]:
!nnUNetv2_train 3 3d_lowres 0 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 1 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 2 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 3 --npz 

In [ ]:
!nnUNetv2_train 3 3d_lowres 4 --npz 

### Train 3d full resolution

In [ ]:
!nnUNetv2_train 3 3d_fullres 0 --npz 

In [ ]:
!nnUNetv2_train 3 3d_fullres 1 --npz 

In [ ]:
!nnUNetv2_train 3 3d_fullres 2 --npz 

In [ ]:
!nnUNetv2_train 3 3d_fullres 3 --npz 

In [ ]:
!nnUNetv2_train 3 3d_fullres 4 --npz 

### Find Best Configuration

In [ ]:
!nnUNetv2_find_best_configuration 3 -c 3d_lowres 


### Prediction

In [ ]:
!nnUNetv2_predict -d Dataset003_HPC -i /dataset/nnUNet_test_data -o /dataset/nnUNet_Prediction_Results/Dataset003_HPC -f  0 1 2 3 4 -tr nnUNetTrainer -c 3d_lowres -p nnUNetPlans

### Post processing 3d lower resolution

In [ ]:
!nnUNetv2_apply_postprocessing -i /dataset/nnUNet_Prediction_Results/Dataset003_HPC -o /dataset/nnUNet_postprocessed/Dataset003_HPC/3d_lowers -pp_pkl_file /dataset/nnUNet_results/Dataset003_HPC/nnUNetTrainer__nnUNetPlans__3d_lowres/crossval_results_folds_0_1_2_3_4/postprocessing.pkl -np 8 -plans_json /dataset/nnUNet_results/Dataset003_HPC/nnUNetTrainer__nnUNetPlans__3d_lowres/crossval_results_folds_0_1_2_3_4/plans.json